# Pydantic AI Demos - Part 1 (Structure)

This notebook demonstrates how pydantic AI supports returning results in a structured form. It starts with the basics
and then goes on with more complicated examples.

## Basic LLM Call

In this example we will just call an LLM and wait for an answer. We use the well known example of asking for the capital of a country.
This just returns a string.


In [3]:
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.openai import OpenAIProvider

BASE_URL="http://127.0.0.1:1234/v1"
LM_STUDIO_MODEL="openai/gpt-oss-20b"

model = OpenAIChatModel(LM_STUDIO_MODEL, provider=OpenAIProvider(BASE_URL))

#Alternative, when only using e.g. OpenAI
# agent = Agent('openai:gpt-5', output_type=str)

agent = Agent(model, output_type=str)

result = await agent.run("What is the capital of Germany?")
result.output


'The capital of Germany is **Berlin**.'



## Some structure to the result

We would like some more information and we also would like it in another form than as string.

So lets create a target object and use this.


In [11]:
from pydantic import BaseModel, Field

class Capital(BaseModel):
    name: str = Field(..., description="The name of the capital")
    country: str = Field(..., description="The country of the capital, as it was asked by the user.")
    num_inhabitants: int = Field(..., description="The number of inhabitants of the capital")

agent = Agent(model, output_type=Capital)

result = await agent.run("What is the capital of Germany?")
hauptstadt = result.output
print(f"name: {hauptstadt.name}, country: {hauptstadt.country}, num_inhabitants: {hauptstadt.num_inhabitants}")

09:10:09.054 agent run
09:10:09.056   chat openai/gpt-oss-20b
name: Berlin, country: Germany, num_inhabitants: 3748000



## Lets add some logging

We are using logfire. A product of pydantic to see what happens during the call.

You would need to setup an account to get the token. There is a free account with a rather large limit.

In [5]:
from dotenv import load_dotenv
import logfire
import os

load_dotenv()
logfire.configure(token=os.getenv("LOGFIRE_TOKEN"))
logfire.info('I am configured!')
logfire.instrument_pydantic_ai()

09:07:42.436 I am configured!


Logfire project URL: 
]8;id=289820;https://logfire-eu.pydantic.dev/joerg-mueller/pydantic-demos\https://logfire-eu.pydantic.dev/joerg-mueller/pydantic-demos]8;;\


In [8]:
# Run the agent again.
result = await agent.run("What is the capital of Japan?")
result.output


09:09:52.220 agent run
09:09:52.222   chat openai/gpt-oss-20b
09:09:53.728   chat openai/gpt-oss-20b


RandomValue(value=37)


## Add a validator

We not only want the result to be structured but also validate, that certain elements have a specific form.
Pydantic AI will do the validation for us and ask the LLM for correction if the result is wrong.

We use an example that forces a retry. We ask for a random number between 0 and 100 but expect it to be always 37.
This forces a retry for demonstration purposes.


In [7]:
from pydantic import model_validator

class RandomValue(BaseModel):
    value: int = Field(..., description="A random value between 0 and 100.")

    @model_validator(mode="after")
    def validate_randomness(self):
        if self.value != 37:
            raise ValueError("The value is not 37. ")
        return self

agent = Agent(model, output_type=RandomValue, system_prompt="You are a random number generator.")

result = await agent.run("Give me a number between 0 and 100!")
result.output

09:08:34.935 agent run
09:08:34.938   chat openai/gpt-oss-20b
09:08:36.217   chat openai/gpt-oss-20b


RandomValue(value=37)